In [1]:
import lightgbm as lgb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
seed = 42

Uso los dataframe de train y test que salen de data prepocesing.

In [3]:
train = pd.read_csv("../datasets/xgb-train.csv")
test = pd.read_csv("../datasets/xgb-test.csv")

In [4]:
train.tail()

,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Opportunity_ID,ASP,ASP_(converted),Month,Delivery_Year,TRF,...,Source _0,Source _1,Source _2,Source _3,Account_Type_0,Account_Type_1,Account_Type_2,Total_Amount_Currency_0,Total_Amount_Currency_1,Total_Amount_Currency_2
16942,1,1,1,1,12801,0.5200,0.58817,12,2016,0,...,1,0,1,1,1,0,0,0,0,1
16943,1,1,1,1,12801,0.5200,0.58817,12,2016,0,...,1,0,1,1,1,0,0,0,0,1
16944,1,1,1,1,12801,0.5200,0.58817,12,2016,0,...,1,0,1,1,1,0,0,0,0,1
16945,1,1,1,1,12802,0.6375,0.63750,12,2016,4,...,1,1,0,1,1,0,0,1,0,0
16946,1,1,1,1,12803,0.6400,0.64000,12,2016,40,...,1,1,0,1,1,0,0,1,0,0


In [5]:
X = train.drop('target', axis=1).values
Y = train['target'].values

X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=seed)

In [6]:
param_grid = {
    'num_leaves': [55, 60, 65],
    'max_depth': [8,10,12],
    'min_gain_to_split':[0.1, 0.2], 
    'max_bin':[50, 100, 150],
    'min_data_in_leaf':[3000, 5000, 7000],
    'bagging_freq':[4,5,6],
    'bagging_fraction':[0.65, 0.7, 0.75],
    'feature_fraction':[0.7]
}

In [7]:
reg = lgb.LGBMRegressor(boosting_type='gbdt',  objective='regression', metric='mae', num_boost_round=5000,
                       verbose=0, learning_rate=0.1)

In [8]:
#gridsearch = GridSearchCV(reg, param_grid, cv=4, scoring = 'neg_mean_absolute_error')
gridsearch = RandomizedSearchCV(reg, param_grid, n_iter=120, n_jobs=3, cv=4, scoring = 'neg_mean_absolute_error')

In [ ]:
%%time
gridsearch.fit(X,Y)

In [ ]:
gridsearch.best_params_

In [ ]:
def get_best_params():
    return {
        'lightgbm':{
            'num_leaves': 65,
            'min_gain_to_split': 0.1,
            'min_data_in_leaf': 1000,
            'max_depth': 10,
            'max_bin': 150,
            'feature_fraction': 0.7,
            'bagging_freq': 6,
            'bagging_fraction': 0.65,
            'learning_rate': 0.05,
            'num_boost_round':10000
        }
    }

In [ ]:
params = get_best_params()['lightgbm']

In [ ]:
params


In [ ]:
%%time
d_train = lgb.Dataset(X_train.values, label=Y_train.values)
d_valid = lgb.Dataset(X_val.values, label=Y_val.values)
watchlist = [d_valid, d_train]
reg = lgb.train(params, train_set = d_train, valid_sets = watchlist, verbose_eval=500)

In [ ]:
Y_pred = reg.predict(X_val.values)

f = np.vectorize(math.exp)
Y_pred = f(Y_pred)
Y_val = f(Y_val.values)
mean_absolute_error(Y_val,Y_pred)

In [ ]:

ids = test_selected.index.values
X_test = test_selected.values
test_predict = reg.predict(X_test)
f = np.vectorize(math.exp)
test_predict = f(test_predict)
escribir_respuesta(ids, test_predict)